# Running an ensemble with different snow interception schemes

First, we have to install the setup, which simply fixes paths in the file manager.

In [2]:
%pylab inline
import os
from pathlib import Path
import pysumma as ps
import pandas as pd
import xarray as xr
from matplotlib import cm
import seaborn as sns
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
sns.set_context('talk')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 12)
year = lambda x, y: slice('{}/10/01'.format(x), '{}/09/30'.format(y))
month = lambda x, y: slice('{}/11/01'.format(x), '{}/12/30'.format(y))

Populating the interactive namespace from numpy and matplotlib


In [3]:
executable = '/pool0/data/andrbenn/summa/bin/summa.exe'
file_manager = './file_manager.txt'

layer_dict = {
    'period': 1, 'instant': 0, 'sum': 0, 
    'mean': 1, 'variance': 0, 'min': 0, 'max': 0
}

var_dict = {'scalarSnowDepth': layer_dict,
            'scalarCanopySnowUnloading': layer_dict,
            'scalarCanopyWat': layer_dict,
            'mLayerTemp': layer_dict,
            'windspd': layer_dict,
            'mLayerDepth': layer_dict,
            'mLayerHeight': layer_dict,  
            'iLayerHeight': layer_dict}

config = {'melt': {
             'decisions': {'snowIncept': 'stickySnow', 'snowUnload': 'meltDripUnload'},
             'output_control': var_dict
         },
         'wind': {
             'decisions': {'snowIncept': 'stickySnow', 'snowUnload': 'windUnload'},
             'output_control': var_dict
         }}

In [4]:
ens = ps.Ensemble(executable, config, file_manager, num_workers=len(config))

/pool0/data/andrbenn/.conda/all/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [5]:
ens.run('local')

In [6]:
ens.simulations['melt'].stdout

"file_suffix is 'melt'.\nfile_master is '/pool0/data/andrbenn/dana_windysnow_test/file_manager.txt'.\ndecisions file =  /pool0/data/andrbenn/dana_windysnow_test/settings/decisions.txt\n   1 simulStart: 2002-10-01 00:00\n   2 simulFinsh: 2003-05-31 00:00\n   3 tmZoneInfo: localTime\n   4 soilCatTbl: ROSETTA\n   5 vegeParTbl: MODIFIED_IGBP_MODIS_NOAH\n   6 soilStress: NoahType\n   7 stomResist: BallBerry\n   8 num_method: itertive\n   9 fDerivMeth: analytic\n  10 LAI_method: monTable\n  11 f_Richards: mixdform\n  12 groundwatr: qTopmodl\n  13 hc_profile: pow_prof\n  14 bcUpprTdyn: nrg_flux\n  15 bcLowrTdyn: zeroFlux\n  16 bcUpprSoiH: liq_flux\n  17 bcLowrSoiH: zeroFlux\n  18 veg_traits: CM_QJRMS1988\n  19 rootProfil: powerLaw\n  20 canopyEmis: difTrans\n  21 snowIncept: stickySnow\n  22 snowUnload: windUnload\n\n\nFATAL ERROR: summa_paramSetup/mDecisions/readoption/cannotFindDecisionIndex[name='snowUnload']\n\n"

In [ ]:
ens.run('local')

In [ ]:
for k, s in ens.simulations.items():
    print(s.status)

In [ ]:
print(list(ens.simulations.values())[-1].stdout[-5600:])

In [ ]:
#ds_jrdn, ds_clm, 
ds_melt, ds_wind = [s.output for s in ens.simulations.values()]

In [ ]:
ts = slice('2001-03-20', '2003-05-31')

In [ ]:
ds_melt['scalarCanopyWat_mean'].sel(time=ts).plot(label='melt')
ds_wind['scalarCanopyWat_mean'].sel(time=ts).plot(label='wind')
plt.legend()

In [ ]:
(3600*ds_melt['pptrate'].sel(time=ts)).plot(label='melt')
(3600*ds_wind['pptrate'].sel(time=ts)).plot(label='wind')
plt.legend()

In [ ]:
ds_melt['scalarSnowDepth_mean'].sel(time=ts).plot(label='melt')
ds_wind['scalarSnowDepth_mean'].sel(time=ts).plot(label='wind')
plt.legend()

In [ ]:
ds_melt['windspd_mean'].sel(time=ts).plot(label='melt')
ds_wind['windspd_mean'].sel(time=ts).plot(label='wind')
plt.legend()

In [ ]:
(ds_melt['airtemp'] - 273.16).sel(time=ts).plot(label='melt')
(ds_wind['airtemp'] - 273.16).sel(time=ts).plot(label='wind')
plt.legend()